# Auto Level Set Liver Tumor Segmentation

In [33]:
import math
import os
import sys

import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [28]:
# Main Folder Dir
fp = 'C:\\Users\\colli\\Desktop\\lesionInterfaces\\\ISMRM_AutoLevelSet'
#     C:\\Users\\colli\\Desktop\\lesionInterfaces\\\ISMRM_AutoLevelSet\\WC-IRB1308014251_008

# Utility variables
splitter = '\\'

# Pandas dataframe to store everything
"""
Shorthand meanings
    - 'vol' = original image volume
    - 'GT' = ground truth segmentation
    - 'MS' = Manual Segmentation
    - 'SC' = Smart Click Segmentation
    - 'LS' = Level Set Segmentation
"""

# Maping file names to respective categories
file_map = {
    'vol' : 'dicom_vol',
    'LM' : 'pred_vol',
    'GT' : 'GT_liver_cyst_vol',
    'MS' : 'DR_liver_cyst_vol',
    'LS' : 'CL_liver_cyst_level_set_vol'} # DR_liver_cyst_smart_click_vol

inv_file_map = {v: k for k, v in file_map.items()}

### Utils

In [29]:
def openImage(fp):
    """
    Open up nifty image and convert to numpy arrays

    Paramaters
    ----------
    fp: string
        The file path to the nifty image

    Returns
    ----------
    nii: nibabel image object
    img: numpy array
    vol: volume of image in cm³
    """
    
    if (not fp) or ('nii' not in fp):
        return

    nii = nib.load(fp)
    sx, sy, sz = nii.header.get_zooms()
    
    return nii, nii.get_fdata(), sx*sy*sz/10

def organizeFiles(fp):
    """
    Organize nifty files into array
    
    Parameters
    ----------
    fp: string
        The file path to the folder containing nifty images

    Returns
    ----------
    arr: array of file paths to numpy images
    """

    arr = []
    for f in os.listdir(fp):
        if 'nii' in f:
            arr.append(os.path.join(fp, f))
    return arr

def organizeFolders(fp):
    """
    Organize folders into array
    
    Paramaters
    ----------
    fp: string
        The file path to the folder containing nifty images

    Returns
    ----------
    arr: array of file paths to folders
    """

    print('DEBUG: list directories -', os.listdir(fp))

    arr = []
    for f in os.listdir(fp):
        if os.path.isdir(os.path.join(fp, f)):
            arr.append(os.path.join(fp, f))
    return arr


### [Chan-Vese Segmentation](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.chan_vese) method - This version is written by Hang Zhang.

In [36]:
# Initial values: mu, nu, max_iter, epison, step = 1, 0.2, 30, 0.1, 0.1

def normMinMax(img, msk=None, p_val=None, ll=0, rr=255):
    """
    Min-Max Normalization

    Parameters
    ----------
    img: numpy array
    msk: numpy array
    p_val: float
    ll: float
    rr: float

    Returns
    ----------
    img: numpy array
    """

    new_img = np.zeros(img.shape)
    if msk is not None:
        min_val = np.min(img[msk > 0])
        max_val = np.max(img[msk > 0])
        new_img = (img - min_val) / (max_val - min_val) * (rr-ll) + ll
    else:
        min_val = np.min(img)
        max_val = np.max(img)
        new_img = ( (img - min_val) / (max_val - min_val) ) * (rr-ll) + ll

    if p_val is not None:
        p_val = (p_val - min_val) / (max_val - min_val) * (rr-ll) + ll

        return new_img, p_val

    return new_img

def argwhere_app(a): # @Jörn Hees's soln: https://stackoverflow.com/questions/48987774/how-to-crop-a-numpy-2d-array-to-non-zero-values
    coords = np.argwhere(a)
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    return x_min, x_max+1, y_min, y_max+1

def iterateChanVese(lsf, img, mu, nu, epison, step):
    """
    Iterate Chan-Vese algorithm
    
    Parameters
    ----------
    lsf: numpy array
    img: numpy array
    mu: float
    nu: float
    epison: float
    step: float

    Returns
    ----------
    lsf: numpy array segmentation
    """

    Drc = (epison / math.pi) / (epison*epison + lsf*lsf)
    Hea = 0.5*(1 + (2 / math.pi) * np.arctan(lsf/epison))
    Iy, Ix = np.gradient(lsf)
    s = np.sqrt(Ix*Ix+Iy*Iy)
    Nx = Ix / (s+1e-6)
    Ny = Iy / (s+1e-6)
    Mxx, Nxx = np.gradient(Nx)
    Nyy, Myy = np.gradient(Ny)
    cur = Nxx + Nyy
    Length = nu*Drc*cur

    Lap = cv2.Laplacian(lsf, -1)
    Penalty = mu*(Lap - cur)

    s1 = Hea*img
    s2 = (1-Hea)*img
    s3 = (1-Hea)
    C1 = (s1).sum() / (Hea).sum()
    C2 = (s2).sum() / (s3).sum()
    CVterm = Drc*(-1 * (img - C1)*(img - C1) + 1 * (img - C2)*(img - C2))

    lsf = lsf + step*(Length + Penalty + CVterm)

    return lsf

def runChanVese2D(img, lsf, mu=1, nu=0.2, max_iter=30, epison=0.1, step=0.1):
    """"
    Parameters
    ----------
    img: numpy array
    lsf: numpy array
    mu: float
    nu: float
    max_iter: int
    epison: float
    step: float

    Returns
    ----------
    lsf: numpy array segmentation
    """

    for _ in range(1, max_iter):
        lsf = iterateChanVese(lsf, img, mu, nu, epison, step)
        # showImgContour(lsf, img)

    return lsf

def runChanVerse(img, msk):
    """
    Run Chan-Vese algorithm on 3D image

    Parameters
    ----------
    img: numpy array
        3D image
        saggital, coronal, *axial*
        CRUCIAL TO HAVE Axial as last dimension

    Returns
    ----------
    lsf: numpy array segmentation
    """

    # TODO: THIS CAN BE OPTIMIZED TO BE MORE EFFICIENT
    sb_x, sb_y, sb_w, sb_h = 0, 0, img.shape[1]*2, img.shape[0]*2 # target segmentation shape
    ss_x, ss_y, ss_w, ss_h = 0, 0, 4, 4 # initial lsf section

    # Initialize segmentation mask
    new_msk = np.zeros(img.shape)

    # Initialize inital lsf
    lsf = np.ones((img.shape[0], img.shape[1]), img.dtype)
    lsf[ss_x:ss_x+ss_w, ss_y:ss_y+ss_h] = -1
    lsf = -lsf
    lsf = lsf[sb_x:sb_x+sb_w, sb_y:sb_y+sb_h]

    # Run Chan-Vese algorithm on each axial slice
    for i in tqdm(range(0, img.shape[2])):

        axial_img = img[:, :, i]
        axial_msk = msk[:, :, i]

        # Check if liver exists on this slide
        if (axial_msk > 0).sum() < 1:
            continue

        norm_img = normMinMax(axial_img)

        temp_msk = runChanVese2D(norm_img, lsf)

        new_msk[:, :, i] = (temp_msk >= 0).astype(float)

    return new_msk

### Iterate through files in a directory

In [37]:
# Dataframe to store time results
df = pd.DataFrame()

# Get folder paths
folder_dir = organizeFolders(fp)

# Get file paths and info
for i in folder_dir:

    # Get File Paths
    file_dir = organizeFiles(i)
    patient_id = i.split(splitter)[-1]
    print('DEBUG: patient id -', patient_id, end = '\t')

    # Get Image and Mask File
    img, msk = None, None
    for j in file_dir:
        if file_map['vol'] in j:
            img = openImage(j)
        elif file_map['LM'] in j:
            msk = openImage(j)
        else:
            print('ERROR: file not declared', j)

    # Perform Segmentation
    if img is not None and msk is not None:
        print('DEBUG: Segmenting', patient_id)
        new_msk = runChanVerse(img[1], msk[1])

        # Invert Mask
        new_msk = 1 - new_msk

        # Cut off not liver regions
        new_msk = new_msk * msk[1]
        new_msk = new_msk.astype(np.uint8)

        # Save Segmentation
        new_msk_fp = os.path.join(i, file_map['LS'] + '.nii.gz')
        new_msk_img = nib.Nifti1Image(new_msk, img[0].affine)
        nib.save(new_msk_img, new_msk_fp)
    else:
        print('ERROR: image or mask not found for', patient_id)
        

    break


DEBUG: list directories - ['WC-IRB1308014251_001', 'WC-IRB1308014251_003', 'WC-IRB1308014251_004', 'WC-IRB1308014251_005', 'WC-IRB1308014251_006', 'WC-IRB1308014251_007', 'WC-IRB1308014251_008', 'WC-IRB1308014251_011', 'WC-IRB1308014251_015', 'WC-IRB1308014251_016', 'WC-IRB1308014251_017', 'WC-IRB1308014251_018', 'WC-IRB1308014251_020', 'WC-IRB1308014251_024_Visit2', 'WC-IRB1308014251_025', 'WC-IRB1308014251_026', 'WC-IRB1308014251_030', 'WC-IRB1308014251_031', 'WC-IRB1308014251_032', 'WC-IRB1308014251_033', 'WC-IRB1308014251_034', 'WC-IRB1308014251_035', 'WC-IRB1308014251_039_Visit1', 'WC-IRB1308014251_039_Visit2', 'WC-IRB1308014251_040', 'WC-IRB1308014251_042', 'WC-IRB1308014251_044', 'WC-IRB1308014251_046', 'WC-IRB1308014251_051', 'WC-IRB1308014251_055', 'WC-IRB1308014251_057', 'WC-IRB1308014251_058', 'WC-IRB1308014251_059', 'WC-IRB1308014251_061', 'WC-IRB1308014251_066_Visit1', 'WC-IRB1308014251_066_Visit2', 'WC-IRB1308014251_071', 'WC-IRB1308014251_082_Visit1', 'WC-IRB1308014251_0

  0%|          | 0/94 [00:00<?, ?it/s]